In [1]:
import numpy as np
from torchvision import datasets, transforms

# Loading transformers
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Converting to numpy for nnlinear
def to_numpy(dataset):
    inputs = []
    labels = []
    for img, label in dataset:
        img = img.view(-1).numpy()  
        inputs.append(img)
        labels.append(label)
    return np.array(inputs), np.array(labels)

X_train, y_train = to_numpy(train_data)
X_test, y_test = to_numpy(test_data)

# Normalize
X_train = X_train / 255.0
X_test = X_test / 255.0


c:\Users\Pradip\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


100%|██████████| 170498071/170498071 [00:09<00:00, 18124296.91it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
from nnlinear import Model, Linear, ReLU, SoftmaxCrossEntropyLoss
from nnlinear.optim import SGD

model = Model([
    Linear(3072, 128),
    ReLU(),
    Linear(128, 10)
])

loss_fn = SoftmaxCrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=0.01)

def train(model, X, y, epochs=10, batch_size=64):
    for epoch in range(epochs):
        indices = np.random.permutation(len(X))
        X, y = X[indices], y[indices]

        for i in range(0, len(X), batch_size):
            xb = X[i:i+batch_size]
            yb = y[i:i+batch_size]

            out = model(xb)
            loss = loss_fn(out, yb)

            model.zero_grad()
            loss_fn.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}: Loss = {loss:.4f}")

train(model, X_train, y_train, epochs=15)


In [ ]:
# Accuracy Rate
def accuracy(model, X, y):
    preds = model(X).argmax(axis=1)
    return (preds == y).mean()

print("Test Accuracy:", accuracy(model, X_test, y_test) * 100, "%")
